In [0]:
# Install all the dependencies in Colab environment i.e. Apache Spark 2.4.4 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
# Setup Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
# Start Spark Session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
#Upload Student_Grades_Data.csv file from local system to remote colab location
from google.colab import files
files.upload()

Saving Student_Grades_Data.csv to Student_Grades_Data.csv


{'Student_Grades_Data.csv': b'Time_to_Study,Grades\r\n1,1.5\r\n5,2.7\r\n7,3.1\r\n3,2.1\r\n2,1.8\r\n9,3.9\r\n6,2.9\r\n12,4.5\r\n11,4.3\r\n2,1.8\r\n4,2.4\r\n8,3.5\r\n13,4.8\r\n9,3.9\r\n14,5\r\n10,4.1\r\n6,2.9\r\n12,4.5\r\n1,1.5\r\n4,2.4\r\n14,5\r\n10,4.1\r\n11,4.3\r\n4,2.4\r\n5,2.7\r\n8,3.5\r\n1,1.5\r\n2,1.8\r\n3,2.1\r\n7,3.1\r\n8,3.5\r\n14,5\r\n7,3.1\r\n8,3.5\r\n1,1.5\r\n2,1.8\r\n3,2.1\r\n4,2.4\r\n5,2.7\r\n6,2.9\r\n7,3.1\r\n8,3.5\r\n9,3.9\r\n10,4.1\r\n11,4.3\r\n12,4.5\r\n13,4.8\r\n14,5\r\n8,3.5\r\n2,1.8\r\n'}

In [0]:
#Loading the Student_Grades_Data.csv file, uploaded in previous step
data = spark.read.csv('Student_Grades_Data.csv', header=True, inferSchema=True)

In [0]:
#Taking a look at data type of each column to see what data types inferSchema=TRUE paramter has set for each column
data.printSchema()

root
 |-- Time_to_Study: integer (nullable = true)
 |-- Grades: double (nullable = true)



In [0]:
#Display first few rows of data
data.show()

+-------------+------+
|Time_to_Study|Grades|
+-------------+------+
|            1|   1.5|
|            5|   2.7|
|            7|   3.1|
|            3|   2.1|
|            2|   1.8|
|            9|   3.9|
|            6|   2.9|
|           12|   4.5|
|           11|   4.3|
|            2|   1.8|
|            4|   2.4|
|            8|   3.5|
|           13|   4.8|
|            9|   3.9|
|           14|   5.0|
|           10|   4.1|
|            6|   2.9|
|           12|   4.5|
|            1|   1.5|
|            4|   2.4|
+-------------+------+
only showing top 20 rows



In [0]:
#Create a Feature array by omitting the last column
feature_cols = data.columns[:-1] 
from pyspark.ml.feature import VectorAssembler
vect_assembler = VectorAssembler(inputCols=feature_cols,outputCol="features")

In [0]:
#Utilize Assembler created above in order to add the feature column
data_w_features = vect_assembler.transform(data)

In [0]:
#Display the data having additional column named features. Had it been multiple linear regression problem, you could see all the
# independent variable values combined in one list
data_w_features.show()

+-------------+------+--------+
|Time_to_Study|Grades|features|
+-------------+------+--------+
|            1|   1.5|   [1.0]|
|            5|   2.7|   [5.0]|
|            7|   3.1|   [7.0]|
|            3|   2.1|   [3.0]|
|            2|   1.8|   [2.0]|
|            9|   3.9|   [9.0]|
|            6|   2.9|   [6.0]|
|           12|   4.5|  [12.0]|
|           11|   4.3|  [11.0]|
|            2|   1.8|   [2.0]|
|            4|   2.4|   [4.0]|
|            8|   3.5|   [8.0]|
|           13|   4.8|  [13.0]|
|            9|   3.9|   [9.0]|
|           14|   5.0|  [14.0]|
|           10|   4.1|  [10.0]|
|            6|   2.9|   [6.0]|
|           12|   4.5|  [12.0]|
|            1|   1.5|   [1.0]|
|            4|   2.4|   [4.0]|
+-------------+------+--------+
only showing top 20 rows



In [0]:
#Select only Features and Label from previous dataset as we need these two entities for building machine learning model
finalized_data = data_w_features.select("features","Grades")

finalized_data.show()

+--------+------+
|features|Grades|
+--------+------+
|   [1.0]|   1.5|
|   [5.0]|   2.7|
|   [7.0]|   3.1|
|   [3.0]|   2.1|
|   [2.0]|   1.8|
|   [9.0]|   3.9|
|   [6.0]|   2.9|
|  [12.0]|   4.5|
|  [11.0]|   4.3|
|   [2.0]|   1.8|
|   [4.0]|   2.4|
|   [8.0]|   3.5|
|  [13.0]|   4.8|
|   [9.0]|   3.9|
|  [14.0]|   5.0|
|  [10.0]|   4.1|
|   [6.0]|   2.9|
|  [12.0]|   4.5|
|   [1.0]|   1.5|
|   [4.0]|   2.4|
+--------+------+
only showing top 20 rows



In [0]:
#Split the data into training and test model with 70% obs. going in training and 30% in testing
train_dataset, test_dataset = finalized_data.randomSplit([0.7, 0.3])

In [0]:
#Peek into training data
train_dataset.describe().show()

+-------+------------------+
|summary|            Grades|
+-------+------------------+
|  count|                31|
|   mean|3.3451612903225802|
| stddev|1.1609877144562206|
|    min|               1.5|
|    max|               5.0|
+-------+------------------+



In [0]:
#Peek into test_dataset
test_dataset.describe().show()

+-------+------------------+
|summary|            Grades|
+-------+------------------+
|  count|                19|
|   mean|3.0210526315789474|
| stddev|1.0036483156566685|
|    min|               1.5|
|    max|               4.8|
+-------+------------------+



In [0]:
#Import Linear Regression class called LinearRegression
from pyspark.ml.regression import LinearRegression

In [0]:
#Create the Linear Regression object named having feature column as features and Label column as Time_to_Study
LinReg = LinearRegression(featuresCol="features", labelCol="Grades")

In [0]:
#Train the model on the training using fit() method.
model = LinReg.fit(train_dataset)

In [0]:
#Predict the Grades using the evulate method
pred = model.evaluate(test_dataset)

In [0]:
#Show the predicted Grade values along side actual Grade values
pred.predictions.show()

+--------+------+------------------+
|features|Grades|        prediction|
+--------+------+------------------+
|   [1.0]|   1.5|1.5663703357117775|
|   [1.0]|   1.5|1.5663703357117775|
|   [2.0]|   1.8|1.8366768043045956|
|   [3.0]|   2.1|2.1069832728974136|
|   [3.0]|   2.1|2.1069832728974136|
|   [4.0]|   2.4| 2.377289741490232|
|   [4.0]|   2.4| 2.377289741490232|
|   [4.0]|   2.4| 2.377289741490232|
|   [7.0]|   3.1| 3.188209147268686|
|   [7.0]|   3.1| 3.188209147268686|
|   [7.0]|   3.1| 3.188209147268686|
|   [8.0]|   3.5|3.4585156158615042|
|   [8.0]|   3.5|3.4585156158615042|
|   [8.0]|   3.5|3.4585156158615042|
|   [9.0]|   3.9|3.7288220844543223|
|  [10.0]|   4.1|3.9991285530471403|
|  [10.0]|   4.1|3.9991285530471403|
|  [12.0]|   4.5| 4.539741490232776|
|  [13.0]|   4.8| 4.810047958825595|
+--------+------+------------------+



In [0]:
#Find out coefficient value
coefficient = model.coefficients
print ("The coefficient of the model is : %a" %coefficient)

The coefficient of the model is : DenseVector([0.2703])


In [0]:
#Find out intercept Value
intercept = model.intercept
print ("The Intercept of the model is : %f" %intercept)

The Intercept of the model is : 1.296064


In [0]:
#Evaluate the model using metric like Mean Absolute Error(MAE), Root Mean Square Error(RMSE) and R-Square
from pyspark.ml.evaluation import RegressionEvaluator
evaluation = RegressionEvaluator(labelCol="Grades", predictionCol="prediction")

# Root Mean Square Error
rmse = evaluation.evaluate(pred.predictions, {evaluation.metricName: "rmse"})
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = evaluation.evaluate(pred.predictions, {evaluation.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = evaluation.evaluate(pred.predictions, {evaluation.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = evaluation.evaluate(pred.predictions, {evaluation.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 0.069
MSE: 0.005
MAE: 0.056
r2: 0.995


In [0]:
#Create Unlabeled dataset  to contain only feature column
unlabeled_dataset = test_dataset.select('features')

In [0]:
#Display the content of unlabeled_dataset
unlabeled_dataset.show()

+--------+
|features|
+--------+
|   [1.0]|
|   [1.0]|
|   [2.0]|
|   [3.0]|
|   [3.0]|
|   [4.0]|
|   [4.0]|
|   [4.0]|
|   [7.0]|
|   [7.0]|
|   [7.0]|
|   [8.0]|
|   [8.0]|
|   [8.0]|
|   [9.0]|
|  [10.0]|
|  [10.0]|
|  [12.0]|
|  [13.0]|
+--------+



In [0]:
#Predict the model output for fresh & unseen test data using transform() method
new_predictions = model.transform(unlabeled_dataset)

In [0]:
#Display the new prediction values
new_predictions.show()

+--------+------------------+
|features|        prediction|
+--------+------------------+
|   [1.0]|1.5663703357117775|
|   [1.0]|1.5663703357117775|
|   [2.0]|1.8366768043045956|
|   [3.0]|2.1069832728974136|
|   [3.0]|2.1069832728974136|
|   [4.0]| 2.377289741490232|
|   [4.0]| 2.377289741490232|
|   [4.0]| 2.377289741490232|
|   [7.0]| 3.188209147268686|
|   [7.0]| 3.188209147268686|
|   [7.0]| 3.188209147268686|
|   [8.0]|3.4585156158615042|
|   [8.0]|3.4585156158615042|
|   [8.0]|3.4585156158615042|
|   [9.0]|3.7288220844543223|
|  [10.0]|3.9991285530471403|
|  [10.0]|3.9991285530471403|
|  [12.0]| 4.539741490232776|
|  [13.0]| 4.810047958825595|
+--------+------------------+

